In [8]:
using Pkg;
Pkg.activate(@__DIR__);
Pkg.instantiate();

  Activating project at `~/Desktop/Work/Research/LEGO-project/LEGO-3D_Dyn`


In [9]:
import ForwardDiff as FD
using LinearAlgebra
using Plots

### Newton's method

In [10]:
function newton_step(func::Function, x0::Vector)::Vector
	# function value at current point
  f = func(x0)
  # check if function is scalar or vector and calculate the Jacobian accordingly
  if length(f) == 1
    J = FD.gradient(func, x0)
    return x0 .- f[1] / J[1]
  else
    J = FD.jacobian(func, x0)
    return x0 - J \ f
  end 
end

function newton_root_finding(func::Function, x0::Vector; tol=1e-6, max_iter=100)::Vector
  x = x0
  for i in 1:max_iter
		# take newton step
    x_new = newton_step(func, x)
    if func(x_new) < tol
      
      return x_new
    end
    x = x_new
  end
	throw("Max iterations reached")
  return x
end

newton_root_finding (generic function with 1 method)

### Quaternions

In [14]:
function hat(v::Vector)::Matrix
  # generate skew symmetric matrix from vector
  return [   0  -v[3]  v[2]; 
           v[3]     0 -v[1];
          -v[2]  v[1]     0]
end

function L(Q::Vector)::Matrix
  # left multiply matrix of quaternion Q
  s = Q[1] # scalar part
  v = Q[2:end] # vector part
  return [s -v'; v hat(v) + s*I]
end

function R(Q::Vector)::Matrix
  # right multiply matrix of quaternion Q
  s = Q[1] # scalar part
  v = Q[2:end] # vector part
  return [s -v'; v hat(v) - s*I]
end

H = [zeros(1,3); I]; # hat map for vectors

T = Diagonal([1; -ones(3)]); # quaternion conjugation matrix

function G(Q::Vector)::Matrix
  # generate attitude jacobian from quaternion Q
  return L(Q) * H
end

function quat2rot(Q::Vector)::Matrix
  # convert quaternion to rotation matrix
  return H' * L(Q) * R(Q) * H
end

quat2rot (generic function with 1 method)

### Dynamics

In [13]:
rigidbody_params = (m=1.0, J=Diagonal([1.0, 1.0, 1.0]), g=9.81) # rigidbody parameters

function linear_kinetic_energy(states::NamedTuple, body_params::NamedTuple)::Float64
  """
  # Calculate linear kinetic energy of the system

  # Arguments 
  - `states`: NamedTuple containing states of the system
  - `body_params`: NamedTuple containing rigidbody parameters

  # Returns
  - `T`: linear kinetic energy of the system
  """

  vᴺ = states.vᴺ
  m = body_params.m
  T = 1/2 * m * vᴺ' * vᴺ # linear kinetic energy
  return T
end

function potential_energy(states::NamedTuple, body_params::NamedTuple)::Float64
  """
  # calculate potential energy of the system

  # Arguments
  - `states`: NamedTuple containing states of the system
  - `body_params`: NamedTuple containing rigidbody parameters

  # Returns
  - `U`: potential energy of the system
  """

  qᴮ = states.qᴮ
  qᴺ = quat2rot(states.ᴺQᴮ) * qᴮ # global orientation
  m, g = body_params.m, body_params.g
  U = m * g * qᴺ[3] # potential energy
  return U
end

function linear_lagrangian(states::NamedTuple, body_params::NamedTuple)::Float64
  """
  # Calculate linear Lagrangian of the system

  # Arguments 
  - `states`: NamedTuple containing states of the system
  - `body_params`: NamedTuple containing rigidbody parameters

  # Returns
  - `L`: Lagrangian of the system
  """

  linear_L = linear_kinetic_energy(states, body_params) - potential_energy(states, body_params) # Lagrangian
  return linear_L
end

# TODO: fix the derivation of the discrete linear lagangian and start formulating the DEL for angular case

function discrete_linear_lagrangian(state_1::NamedTuple, state_2::NamedTuple, body_params::NamedTuple, h::Float64)::Float64
  """
  # Calculate discrete Lagrangian of the system

  # Arguments
  - `state_1`: NamedTuple containing states of the system at time t
  - `state_2`: NamedTuple containing states of the system at time t + h
  - `body_params`: NamedTuple containing rigidbody parameters
  - `h`: time step

  # Returns
  - `Ld`: discrete Lagrangian of the system using midpoint integration
  """

  qᴮ_midpoint = 1/2 * (state_1.qᴮ + state_2.qᴮ) # midpoint position
  vᴺ_midpoint = (state_2.qᴮ - state_1.qᴮ) / h # velocity
  


  interpolated_states = (qᴮ=qᴮ_midpoint, vᴺ=vᴺ_midpoint, ᴺQᴮ=state_1.ᴺQᴮ, ωᴮ=state_1.ωᴮ) # interpolated states
  Ld = h * lagrangian(interpolated_states, body_params) # discrete Lagrangian
  return Ld

end

lagrangian (generic function with 2 methods)